In [351]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lungtesttxt/lung_test.txt
/kaggle/input/lungtxt/lung.txt


In [352]:
import pandas as pd
import numpy as np

In [353]:
df = pd.read_csv('/kaggle/input/lungtxt/lung.txt',header = None)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,1,-0.953518,-0.433265,-0.926334,-0.878343,-0.940477,-0.881761,-0.555303,-0.866727,-0.339190,...,0.111509,-0.842061,-0.984096,-0.641238,-0.611943,-0.852940,-0.891501,-0.737089,-0.879686,-0.998865
1,1,-0.976293,-0.750423,-0.941069,-0.980535,-0.994048,-0.925679,-0.844926,-0.942945,-0.778852,...,0.256599,-0.918326,-0.997074,-0.912676,-0.895525,-0.985294,-0.945752,-0.768386,-0.942223,-0.999525
2,1,-0.663669,-0.491751,-0.893186,-0.854013,-0.934524,-0.842909,-0.792472,-0.907733,-0.490566,...,0.184607,-0.844687,-0.969846,-0.038363,-0.425380,-0.887256,-0.938516,-0.729780,-0.903817,-0.979734
3,1,-0.852158,-0.730928,-0.697973,-0.970803,-0.928571,-0.940880,-0.621438,-0.958102,-0.739355,...,0.496361,-0.900450,-0.989313,-0.681212,-0.701494,-0.936275,-0.978300,-0.814291,-0.920470,-0.992083
4,1,-0.987727,-0.858310,-0.985267,-0.975669,-1.000000,-0.988176,-0.908780,-0.938043,-0.861778,...,0.565058,-0.545379,-0.996183,-0.910572,-0.880595,-0.990196,-1.000000,-0.854985,-0.555620,-0.988864


In [354]:
df_clean = df.dropna(1,'any')

In [355]:
pel_class = np.unique(df_clean[0])
pel_class

array([0, 1])

In [356]:
pel_value = [sum(df_clean[0]==pel_class[0]),
             sum(df_clean[0]==pel_class[1])]
pel_value

[304, 328]

In [357]:
y = df_clean[0].values
X = df_clean.drop([0],axis = 1).values

In [358]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X, y = sm.fit_resample(X, y)

In [359]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [360]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train,y_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [361]:
from sklearn.decomposition import PCA
fr = PCA(n_components=65)
fr.fit(X_train_norm)
X_train_norm = fr.transform(X_train_norm)
X_test_norm = fr.transform(X_test_norm)

In [362]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
params = {
    'C' : [1,2,4,8,16,32], # High C = Overfitting
    'gamma' : [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32] # High gamma = Overfitting
}
clf = GridSearchCV(SVC(),params, cv=10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

Best params : {'C': 32, 'gamma': 0.0625}
10CV accuracy : 74.05660377358491


In [363]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm)
target_names = ['negative', 'positive']
sum(y_test == y_predict)/len(y_test)

0.8106060606060606

In [364]:
X_test_norm.shape

(132, 65)

In [365]:
df_t = pd.read_csv('/kaggle/input/lungtesttxt/lung_test.txt',header = None)


In [366]:
yt = df_t[0].values
Xt = df_t.drop([0],axis = 1).values

In [367]:
X_test_norm= scaler.transform(Xt)

In [368]:
X_test_norm= fr.transform(X_test_norm)

In [369]:
y_predict = clf.predict(X_test_norm)
target_names = ['negative', 'positive']
sum(yt == y_predict)/len(yt)

0.6470588235294118